<a href="https://colab.research.google.com/github/kiyong21c/pytorch_tutorial/blob/main/20220706_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# logistic regression
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 0) Prepare data
bc = datasets.load_breast_cancer() # sklearn를 통해 불러온 데이터셋
X, y = bc.data, bc.target   # X.shape : (569, 30), y.shape : (569,)

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) # X_train으로 훈련하여 X_test에도 적용

# numpy(array) → torch(tensor)
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# y_train/test 1차원 배열 → 2차원 배열(모델에 사용되기 위함)
y_train = y_train.view(y_train.shape[0], 1) # view() : 넘파이의 reshape()과 같다
y_test = y_test.view(y_test.shape[0], 1) # torch.size([114]) → torch.size([114, 1])

# 1) Model
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    # nn.Module을 상속받으면서 사용자정의 모델 만들시 해야할 두가지

    # 1)__init__()재정의(오버라이딩)
    def __init__(self, n_input_features):
        # super(Model, self).__init__()
        super().__init__()   # # nn.Module 상속
        self.linear = nn.Linear(n_input_features, 1) # 사용될 층 생성, 1)활성화 함수를 층으로 만들어도 되고, 2)forward에 적용해도 됨

    # 2)forward 모듈 재정의(오버라이딩)
    def forward(self, x):   # nn.Module 클래스에 있는 forward 모듈을 재정의(오버라이딩) 해야함
    # 모델생성 후 self.forward() 하지 않아도 nn.Module의 영향으로 자동으로 호출되는 메서드
        y_pred = torch.sigmoid(self.linear(x))  # torch가 제공하는 sigmoid()
        return y_pred

model = Model(n_features) # 보통은 input_features, output_features를 파라미터로 받지만, 이경우 output_features가 1이기 때문에 모델 class 정의할 때 부터 생략
# 파라미터 확인
w, b = model.parameters()
print('최초 w =',w[0], '최초 b =',b[0])

# 2) Loss and optimizer
num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss() # Binary Classification Entrophy 손실함수 : 마지막레이어의 노드가 1개인 이진분류에 주로 사용됨(이경우 암진단결과)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # 옵티마이저에 모델의 파라미터(w, b), learning_rate 전달 

# 3) Training loop
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train) # 모델에 훈련데이터 입력 → 예측값
    loss = criterion(y_pred, y_train) # 손실함수(예측값, 실제값) → 손실값
    # print(loss) # 1.0303 : 스칼라값

    # Backward pass and update
    loss.backward() # dloss/dw, dloss/db 계산됨
    optimizer.step() # optimizer 내부 로직에 의해 모델의 파라미터(w, b) 갱신됨

    # zero grad before new step
    optimizer.zero_grad() # dloss/dw, dloss/db 초기화(0)

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

# 파라미터 확인
w, b = model.parameters()
print('최종 w =',w[0], '최종 b =',b[0]) # 0번째 인덱스가 w,b 각각의 값
print(w.shape, b.shape) # torch.Size([1, 30]) torch.Size([1])

# 1.훈련 데이터셋으로 훈련된(w, b 갱신완료) 모델로 테스트 데이터의 타깃값 예측
# 2.테스트 데이터의 타깃값(예측)과 실제 타깃값으로 정확도(accuracy) 확인
with torch.no_grad():   # backward() 메서드 사용안하는데? no_grad()해야함?, w,b갱신안하는데..
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round() # 반올림
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    # a.eq(b).sum() : (a == b).sum(), 같으면 더해라
    print(f'accuracy: {acc.item():.4f}')

최초 w = tensor([ 0.0029, -0.1791, -0.1629,  0.1317,  0.0757, -0.1802,  0.0287, -0.1818,
        -0.0854, -0.1718, -0.0949, -0.1478, -0.0415, -0.0360,  0.0534,  0.0488,
         0.0199, -0.0316,  0.1676, -0.0130, -0.0985,  0.0291, -0.0802,  0.0003,
         0.1036, -0.0852,  0.0453, -0.0917, -0.0523,  0.1685],
       grad_fn=<SelectBackward0>) 최초 b = tensor(0.0439, grad_fn=<SelectBackward0>)
epoch: 10, loss = 0.4160
epoch: 20, loss = 0.3691
epoch: 30, loss = 0.3349
epoch: 40, loss = 0.3089
epoch: 50, loss = 0.2882
epoch: 60, loss = 0.2713
epoch: 70, loss = 0.2571
epoch: 80, loss = 0.2451
epoch: 90, loss = 0.2347
epoch: 100, loss = 0.2256
최종 w = tensor([-0.1152, -0.2415, -0.2822,  0.0173,  0.0095, -0.2622, -0.0745, -0.3040,
        -0.1272, -0.1539, -0.1766, -0.1228, -0.1189, -0.1208,  0.0562,  0.0319,
         0.0107, -0.0696,  0.1805,  0.0029, -0.2286, -0.0476, -0.2094, -0.1215,
         0.0036, -0.1774, -0.0605, -0.2241, -0.1366,  0.1120],
       grad_fn=<SelectBackward0>) 최종 b = tenso